In [1]:
from src.backend.retrieval.file_reader import read_csv
from src.backend.retrieval.pipeline import TwoStagePipeline, preprocess_text_column

In [2]:
documents = read_csv('data/processed/all_data_cleaned.csv', text_column='cleaned_text')
type(documents)
documents

0         Approximately 100km long firebreaks have been ...
1                                          God bless you...
2         RT : Cracked wine casks, damaged historical  b...
3         I'm really just excited for new undies and pin...
4         Rescue effort expands in India, Pakistan as fl...
                                ...                        
212761        Tips to avoid flood damaged cars after Harvey
212762    you don't have to say that twice... Over here ...
212763    RT : We stand with  in the aftermath of the ea...
212764             Quake: M 5.1, southern East Pacific Rise
212765    Of to do skydive with   landing on a beach can...
Name: cleaned_text, Length: 212766, dtype: object

In [3]:
documents = preprocess_text_column(documents)

Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Error processing line: Approximately 100km long firebreaks have been constructed by Indonesia and elsewhere.. Error: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.
Error processing line: God bless you.... Error: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.
Error processing line: RT : Cracked wine casks, damaged historical  buildings and coffee shops. This Napa earthquake is the biggest first world disaster â€¦. Error: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.
Error processing line: I'm really just excited for new undies and pinkberry. Error: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.
Error processing line: Rescue effort expands in India, Pakistan as flood death toll tops 350. Error: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.
Error processing line: RT : I hope everyone in Chile st

KeyboardInterrupt: 